In [13]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from scipy.stats import skew
import numpy as np
import seaborn as sns
import glob
import re

In [14]:
!rm -r Results
!g++ Proyect.cpp -o ICDF.out
!mkdir Results

In [ ]:
#Estamos trabajando con los datos North
!./ICDF.out

In [ ]:
#Lectura Alternativa del DF
DF = pd.read_csv('./Results/Easth_Complete_trend_analysis.dat', sep = ' ')

In [ ]:
DF.head()

In [ ]:
#Sacamos el N mas frecuente
N_most_frequent = DF.groupby('Iteration').max().reset_index()[['Iteration','#Autos']]
N = N_most_frequent["#Autos"].mode()[0]
iterations = N_most_frequent[N_most_frequent['#Autos'] == N].Iteration.unique()

In [ ]:
DF.groupby('Iteration').max().reset_index()[['Iteration','#Autos']]['#Autos'].value_counts()

In [8]:
#Seleccionamos todos los procesos que contengan al N mas frecuente
DF_ = DF[DF.Iteration.isin(iterations)]

In [9]:
#Revisamos las estadisticas de los resultados 
stats = pd.pivot_table(DF_, index = 'Iteration', values = 'Delta_t', aggfunc = [np.mean,np.std, np.median, skew])

In [10]:
stats.sort_values(('std','Delta_t'))

,mean,std,median,skew
,Delta_t,Delta_t,Delta_t,Delta_t
Iteration,,,,
41776,48.795543,34.206090,47.23360,0.572021
3917,48.762702,34.354220,43.31060,0.494001
22375,48.695789,34.573736,46.09990,0.536267
39964,48.970215,35.430696,45.12685,0.481503
20868,48.807419,35.865663,45.43405,0.413101
...,...,...,...,...
12651,49.608199,48.310906,25.65035,0.627390
19516,49.980459,48.470411,28.75665,0.520358


In [11]:
#Seleccionamos una simulacion que tenga resultados mas cercanos a los reales

In [12]:
DF_ = DF_[DF_.Iteration.isin([13028])]

In [ ]:
#Hacemos el trend analysis de la region North
#Ajustamos Minimos cuadrados
x = np.array(DF_['#Autos']).reshape(-1, 1)
y = np.array(DF_['Delta_t']).reshape(-1, 1)
linreg = LinearRegression().fit(x,y)

In [ ]:
print('Intercept: {}'.format(linreg.intercept_))
print('Coef: {}'.format(linreg.coef_))

In [ ]:
plt.scatter(x,y)
ax = plt.gca()
ax.set_xlabel('Vehicle Count')
ax.set_ylabel('Time Intervals')
plt.plot(x,linreg.intercept_+linreg.coef_*x, color='r')

# NO BORRAR

#Creamos lista de DF
DF = pd.DataFrame()
for i in glob.glob('./Results/*.dat'):
    data = pd.read_csv(i, sep = " ")
    data['Iteration'] = int(re.findall(r'\d+',i)[0])
    DF =  pd.concat([DF,data],ignore_index=True)